In [1]:
import numpy as np
import pandas as pd
import mlflow
import warnings
import pickle
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import ticker
from statsmodels.tsa.stattools import adfuller, acf, pacf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.losses import Huber
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [2]:
# sns.set()
%matplotlib inline
warnings.filterwarnings("ignore")

In [22]:
import sys, os
from time import gmtime, strftime

In [4]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from file_handler import FileHandler

**Preprocessing**

We have alreadyseen which features influence sales in the exploratory section. And we have used many of the columns for the 4 machine learning models used as seen in the scripts. But for LSTM only the date and sales columns will be used.

In [5]:
file_handler = FileHandler()
train_sales = file_handler.read_csv("../data/train.csv")

In [6]:
data = train_sales.groupby("Date").agg({"Sales": "mean"})
data

,Sales
Date,
2013-01-01,87.284560
2013-01-02,6233.030493
2013-01-03,5693.112108
2013-01-04,5954.218834
2013-01-05,5337.751570
...,...
2015-07-27,9602.952466
2015-07-28,8174.953363
2015-07-29,7623.284305


In [7]:
scaler = StandardScaler()
scaled_array = scaler.fit_transform(data)
data['DataScaled'] = scaled_array

In [8]:
SIZE = len(data.DataScaled)
WINDOW_SIZE = 48
BATCH_SIZE = SIZE - WINDOW_SIZE * 2
EPOCHS = 200

In [9]:
DateTrain = data.index.values[0:BATCH_SIZE]
DateValid = data.index.values[BATCH_SIZE:]
XTrain = data.DataScaled.values[0:BATCH_SIZE].astype('float32')
XValid = data.DataScaled.values[BATCH_SIZE:].astype('float32')

# Obtain shapes for vectors of size (,1) for dates series

DateTrain = np.reshape(DateTrain, (-1, 1))
DateValid = np.reshape(DateValid, (-1, 1))

print("Shape of the training set date series: ", DateTrain.shape)
print("Shape of the validation set date series: ", DateValid.shape)
print()
print("Shape of the training set logarithm of sales series: ", XTrain.shape)
print("Shape of the validation set logarithm of sales series in a stateless LSTM: ", XValid.shape)

Shape of the training set date series:  (846, 1)
Shape of the validation set date series:  (96, 1)

Shape of the training set logarithm of sales series:  (846,)
Shape of the validation set logarithm of sales series in a stateless LSTM:  (96,)


In [10]:
tf.random.set_seed(1234)
# add extra dimension
series = tf.expand_dims(XTrain, axis=-1)
series.shape

TensorShape([846, 1])

In [11]:
# create tensor from each individual element
dataset = tf.data.Dataset.from_tensor_slices(series)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(1,), dtype=tf.float32, name=None)>

In [12]:
# takes a window_size + 1 chunk from the slices
dataset = dataset.window(WINDOW_SIZE + 1, shift=1, drop_remainder=True)

In [13]:
# Example of Window
datasetEx = tf.data.Dataset.from_tensor_slices(tf.range(10))
datasetEx = datasetEx.window(5, shift=1, drop_remainder=True)
for window in datasetEx:
    print([elem.numpy() for elem in window])

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


In [14]:
dataset = dataset.flat_map(lambda window: window.batch(WINDOW_SIZE + 1))

In [15]:
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))

In [16]:
dataset = dataset.batch(BATCH_SIZE).prefetch(1)

In [17]:
def windowed_dataset(series, window_size=WINDOW_SIZE, batch_size=BATCH_SIZE):
  series = tf.expand_dims(series, axis=-1)
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [18]:
DatasetTrain = windowed_dataset(XTrain)
DatasetVal = windowed_dataset(XValid)

**_Model Training_**

In [19]:
model = Sequential()
model.add(LSTM(8, input_shape=[None, 1], return_sequences=True))
model.add(LSTM(4, input_shape=[None, 1]))
model.add(Dense(1))
model.compile(loss=Huber(), optimizer='adam')

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, None, 8)        │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 4)              │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 533 (2.08 KB)

 Trainable params: 533 (2.08 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
mlflow.set_experiment("LSTM")
mlflow.tensorflow.autolog()
history = model.fit(DatasetTrain, epochs=EPOCHS, validation_data=DatasetVal, verbose=1)

2025/01/13 12:11:58 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '52951095a9ef4e4ea04a3aeaace466e8', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/200
      1/Unknown 5s 5s/step - loss: 0.4138

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 0.4138 - val_loss: 0.4014
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - loss: 0.4131

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step - loss: 0.4131 - val_loss: 0.4006
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - loss: 0.4126

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step - loss: 0.4126 - val_loss: 0.3997
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step - loss: 0.4120

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step - loss: 0.3133 - val_loss: 0.2466
Epoch 197/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step - loss: 0.3115

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step - loss: 0.3115 - val_loss: 0.2442
Epoch 198/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step - loss: 0.3097

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step - loss: 0.3097 - val_loss: 0.2420
Epoch 199/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - loss: 0.3080

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step - loss: 0.3080 - val_loss: 0.2400
Epoch 200/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step - loss: 0.3063

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step - loss: 0.3063 - val_loss: 0.2380
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/st ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step


In [25]:
time = strftime("%Y-%m-%d_%H-%M-%S", gmtime())  # Use '_' for date-time separation and '-' for time separation
model.save(f'../models/LSTM_sales_{time}.keras')  # Native Keras format
